In [1]:
from typing import TypedDict, Annotated, Sequence
from typing_extensions import TypedDict

from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import (
    BaseMessage,
    HumanMessage,
    AIMessage,
    ToolMessage,
    FunctionMessage,
)
from langchain_openai import ChatOpenAI
from langgraph.graph import END, StateGraph
from langgraph.graph.message import add_messages
from langchain.tools import BaseTool, StructuredTool, Tool, tool
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain import hub
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

# Your further code goes here

In [5]:
import os
import io
import pandas as pd
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from docx import Document
from PyPDF2 import PdfReader
from tqdm import tqdm

# Set up Google Drive API
SCOPES = ["https://www.googleapis.com/auth/drive"]
SERVICE_ACCOUNT_FILE = "google_sheet_key.json"

credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES
)

service = build("drive", "v3", credentials=credentials)

# Define the folder ID
folder_id = "11IqKFcX-zbXJJoqWNxZmW8bGtaKqJNh_"  # Replace with your actual folder ID

# Create a directory for downloaded files
download_dir = "downloaded_docs"
os.makedirs(download_dir, exist_ok=True)

# Function to download a file from Google Drive


def download_file(file_id, file_name):
    request = service.files().get_media(fileId=file_id)
    fh = io.BytesIO()
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()
    file_path = os.path.join(download_dir, file_name)
    with open(file_path, "wb") as f:
        f.write(fh.getbuffer())
    return file_path


# Function to extract text from a .docx file


def extract_text_from_docx(file_path):
    doc = Document(file_path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return "\n".join(full_text)


# Function to extract text from a .pdf file


def extract_text_from_pdf(file_path):
    pdf_text = []
    with open(file_path, "rb") as f:
        reader = PdfReader(f)
        for page in reader.pages:
            pdf_text.append(page.extract_text())
    return "\n".join(pdf_text)


# Get the list of files in the folder
results = (
    service.files()
    .list(q=f"'{folder_id}' in parents", pageSize=100, fields="files(id, name)")
    .execute()
)

items = results.get("files", [])

# Initialize an empty list to hold the data
data = []

# Loop through the files and download, then extract text with progress bar
for item in tqdm(items, desc="Processing files"):
    file_id = item["id"]
    file_name = item["name"]
    local_file_path = download_file(file_id, file_name)

    if file_name.endswith(".docx"):
        text = extract_text_from_docx(local_file_path)
    elif file_name.endswith(".pdf"):
        text = extract_text_from_pdf(local_file_path)
    else:
        continue

    data.append({"File Name": file_name, "Text": text})

    # Clean up the downloaded file
    os.remove(local_file_path)

# Create a DataFrame from the data
df = pd.DataFrame(data)

Processing files: 100%|██████████| 19/19 [00:26<00:00,  1.40s/it]

                             File Name  \
0                Рязанцев Евгений.docx   
1          Славин Роман Михайлович.pdf   
2                  Пастухов Игорь.docx   
3                  Богачев Сергей.docx   
4              Карандашов Алексей.docx   
5                Макряшин Геогрий.docx   
6                   Карлов Сергей.docx   
7                    Антонов Иван.docx   
8               Сотниченко Андрей.docx   
9     Бунеско Андрей Александрович.pdf   
10          Шестопалов Константин.docx   
11               Полушкин Виталий.docx   
12                    Ус Владимир.docx   
13  Клыковский Вячеслав Витальевич.pdf   
14                 Биленко Руслан.docx   
15              Мазепин Станислав.docx   
16             Буханцев Александр.docx   
17               Полостюк Николай.docx   
18                Девицын Евгений.docx   

                                                 Text  
0   \nОпыт работы\nКлючевые навыки\n• Ответственно...  
1   Славин Роман Михайлович\nМужчина, 29 лет, р

In [6]:
df

,File Name,Text
0,Рязанцев Евгений.docx,\nОпыт работы\nКлючевые навыки\n• Ответственно...
1,Славин Роман Михайлович.pdf,"Славин Роман Михайлович\nМужчина, 29 лет, роди..."
2,Пастухов Игорь.docx,\nОпыт работы\nКлючевые навыки\n• Active Direc...
3,Богачев Сергей.docx,\nОпыт работы\nКлючевые навыки\n• Администриро...
4,Карандашов Алексей.docx,\nОпыт работы\n\nДекабрь.2022 —н.в.\tРозничная...
5,Макряшин Геогрий.docx,\nОпыт работы\nКлючевые навыки\n• Исполнительн...
6,Карлов Сергей.docx,\nОпыт работы\nКлючевые навыки\n• Знания уровн...
7,Антонов Иван.docx,\nОпыт работы\nКлючевые навыки\n• MS Windows 2...
8,Сотниченко Андрей.docx,\nОпыт работы\nКлючевые навыки\n• Linux ...
9,Бунеско Андрей Александрович.pdf,"Бунеско Андрей Александрович\nМужчина, 35 лет,..."


In [10]:
import importlib

module = importlib.import_module("gsheet_handler")
importlib.reload(module)
ggl = module.GSheetHandler("google_sheet_key.json")

In [11]:
ggl.update_gsheet_with_df(
    df,
    "https://docs.google.com/spreadsheets/d/18zb9tDE4DiGrMbRt4FMbW7GW6VRzxYPwMr5S9BGJS-I/edit?usp=sharing",
    0,
)